# Conversion Rate Dashboard

## Conexión y librerías

In [1]:
import sys
sys.path.append('../')
import conexion_database

No Laptop Tandamos
Spartan PC


In [2]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np

# Para guardar los archivos con la fecha de hoy
from datetime import datetime, date, timedelta

# Limpieza de query
from clean_query import clean_query

# La función de weeknum es equivalente a la de WEEKNUM en Excel, modo 1, en la que se basa Alejo
from calweek import weeknum

In [3]:
# Conexión con Google Sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import json

scopes = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive'
]
credentials = ServiceAccountCredentials.from_json_keyfile_name("../gsheets_key.json", scopes) #access the json key you downloaded earlier 
client = gspread.authorize(credentials) # authenticate the JSON key with gspread

## Carga de data

**Gráfica - Conversion rate evolution**
* Fecha de conversión: Después de agrupar por usuario, es el MIN de contribution_amount_date en TTAUC, siempre y cuando contribution_number = 1 y contribution_amount_total no es nulo.
* Fecha de afiliación: el campo más grande entre app_user_onboarding_approved_date y terms_conditions_accepted_date TAUO.
* Cohorte por fecha de afiliación: cálculo manual con base en fecha de afiliación

**Gráfica - Users that Activated a Tanda before converting**
* Fecha de activación de tanda: hay que revisar insert_date en TTAU para ver cuándo se creó una tanda. A su vez, para que se considere activa, la tanda debió haber sido creada por un usuario afiliado (ver que app_user_onboarding_approved_date y terms_conditions_accepted_date TAUO no estén nulos) o que el usuario se haya afiliado antes de cancelar la tanda (ver fecha de afiliación y tanda_app_user_cancelled_date TTAU)
* Fecha de conversión: ver arriba
* Cohorte por fecha de afiliación: ver arriba


**Gráfica - Users that Cancelled before converting (affiliated only)**
* Fecha de cancelación de tanda: ver tanda_app_user_cancelled_date TTAU. Además, filtrar solo por los usuarios afiliados y las cancelaciones posteriores a su fecha de afiliación. 
* Fecha de conversión: ver arriba
* Cohorte por fecha de afiliación: ver arriba


### Data conversion-rate-evolution

In [4]:
# Redactar SQL query
query = """
SELECT tau.id as usuario_id,
	name, 
	id_tanda,
	app_user_onboarding_approved_date AS onboarding_approved_date, 
	terms_conditions_accepted_date AS terms_accepted_date,
	user_tandas.fecha_convertido
FROM tdm_app_user AS tau 
	JOIN tdm_app_user_onboarding AS tauo ON tau.id = tauo.app_user_id	
	LEFT JOIN 
	( 	SELECT 
			ttau.app_user_id AS ttau_app_user_id,
			ttau.id as id_tanda,
			MIN(contribution_amount_date) AS fecha_convertido
		FROM tdm_tanda_app_user_contribution as ttauc
		JOIN tdm_tanda_app_user AS ttau ON ttau.id = ttauc.tanda_app_user_id
		WHERE contribution_number = 1 
			AND contribution_amount_total IS NOT NULL
		GROUP BY ttau.app_user_id		
	) AS user_tandas ON tau.id = user_tandas.ttau_app_user_id
	WHERE tau.id NOT IN (7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 41, 90, 188, 369, 636, 929, 1574, 1788, 2394, 2440, 2432, 2442, 2443, 2444, 2445, 2447, 2446, 2450, 2451, 2452,  2454, 2455, 2477, 3227, 4773, 4776, 4816, 7572, 12912, 13697, 19651)
    AND app_user_onboarding_approved_date IS NOT NULL
	AND terms_conditions_accepted_date IS NOT NULL
ORDER BY usuario_id, id_tanda
;
"""
query = clean_query(query)
print(query)

SELECT tau.id as usuario_id, name, id_tanda, app_user_onboarding_approved_date AS onboarding_approved_date, terms_conditions_accepted_date AS terms_accepted_date, user_tandas.fecha_convertido FROM tdm_app_user AS tau JOIN tdm_app_user_onboarding AS tauo ON tau.id = tauo.app_user_id LEFT JOIN ( SELECT ttau.app_user_id AS ttau_app_user_id, ttau.id as id_tanda, MIN(contribution_amount_date) AS fecha_convertido FROM tdm_tanda_app_user_contribution as ttauc JOIN tdm_tanda_app_user AS ttau ON ttau.id = ttauc.tanda_app_user_id WHERE contribution_number = 1 AND contribution_amount_total IS NOT NULL GROUP BY ttau.app_user_id ) AS user_tandas ON tau.id = user_tandas.ttau_app_user_id WHERE tau.id NOT IN (7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 41, 90, 188, 369, 636, 929, 1574, 1788, 2394, 2440, 2432, 2442, 2443, 2444, 2445, 2447, 2446, 2450, 2451, 2452, 2454, 2455, 2477, 3227, 4773, 4776, 4816, 7572, 12912, 13697, 19651)   AND app_user_onboarding_approved_date IS NOT NULL AND terms_conditions_accept

In [ ]:
# Cargar query a un DataFrame
df_conversion_rate = pd.read_sql_query(query, conexion_database.engine)
print(df_conversion_rate.shape)
df_conversion_rate.head(4)

In [6]:
# Calcular columna con fecha de afiliación
df_conversion_rate['fecha_afiliacion'] = df_conversion_rate[['onboarding_approved_date', 'terms_accepted_date']].max(axis=1, skipna=False)

# Agregar cohorte de afiliación (para ello uso la función weeknum() que imita WEEKNUM de Excel)
df_conversion_rate['affiliation_year'] = df_conversion_rate.fecha_afiliacion.dt.year
df_conversion_rate['affiliation_week'] = df_conversion_rate.fecha_afiliacion.apply(weeknum).astype('int')

# Transformar fecha_convertido a fecha
df_conversion_rate['fecha_convertido'] = pd.to_datetime(df_conversion_rate['fecha_convertido'], format = "%Y-%m-%d")

In [ ]:
df_conversion_rate[df_conversion_rate.fecha_convertido > "2022-04-10"]

### Data Activated y cancelled tanda before converting

In [8]:
# Redactar SQL query
query = """
SELECT tau.id,
	name,
	app_user_onboarding_approved_date AS onboarding_approved_date, 
	terms_conditions_accepted_date AS terms_accepted_date,
	tanda_name,
	ttau.insert_date AS fecha_creacion_tanda,
	tanda_app_user_cancelled_date AS fecha_cancelacion_tanda,
	fecha_convertido
FROM tdm_tanda_app_user AS ttau
	JOIN tdm_app_user AS tau ON ttau.app_user_id = tau.id
	JOIN tdm_app_user_onboarding AS tauo ON tauo.app_user_id = tau.id
	LEFT JOIN 
	(
		SELECT ttau.app_user_id as usuario_id,
			MIN(contribution_amount_date) AS fecha_convertido
		FROM tdm_tanda_app_user_contribution as ttauc
			JOIN tdm_tanda_app_user AS ttau ON ttau.id = ttauc.tanda_app_user_id
		WHERE contribution_number = 1 
			AND contribution_amount_total IS NOT NULL
		GROUP BY usuario_id
	) AS fechas_convertidos ON fechas_convertidos.usuario_id = tau.id
WHERE app_user_onboarding_approved_date IS NOT NULL
	AND terms_conditions_accepted_date IS NOT NULL
	AND tau.id NOT IN (7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 41, 90, 188, 369, 636, 929, 1574, 1788, 2394, 2440, 2432, 2442, 2443, 2444, 2445, 2447, 2446, 2450, 2451, 2452,  2454, 2455, 2477, 3227, 4773, 4776, 4816, 7572, 12912, 13697, 19651)
ORDER BY tau.id
;
"""
query = clean_query(query)
print(query)

SELECT tau.id, name, app_user_onboarding_approved_date AS onboarding_approved_date, terms_conditions_accepted_date AS terms_accepted_date, tanda_name, ttau.insert_date AS fecha_creacion_tanda, tanda_app_user_cancelled_date AS fecha_cancelacion_tanda, fecha_convertido FROM tdm_tanda_app_user AS ttau JOIN tdm_app_user AS tau ON ttau.app_user_id = tau.id JOIN tdm_app_user_onboarding AS tauo ON tauo.app_user_id = tau.id LEFT JOIN ( SELECT ttau.app_user_id as usuario_id, MIN(contribution_amount_date) AS fecha_convertido FROM tdm_tanda_app_user_contribution as ttauc JOIN tdm_tanda_app_user AS ttau ON ttau.id = ttauc.tanda_app_user_id WHERE contribution_number = 1 AND contribution_amount_total IS NOT NULL GROUP BY usuario_id ) AS fechas_convertidos ON fechas_convertidos.usuario_id = tau.id WHERE app_user_onboarding_approved_date IS NOT NULL AND terms_conditions_accepted_date IS NOT NULL AND tau.id NOT IN (7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 41, 90, 188, 369, 636, 929, 1574, 1788, 2394, 2440,

In [ ]:
# Cargar query a un DataFrame
df_activated_tanda = pd.read_sql_query(query, conexion_database.engine, parse_dates={'fecha_convertido':'%Y-%m-%d'})
print(df_activated_tanda.shape)
df_activated_tanda.head(4)

In [10]:
# Calcular columna con fecha de afiliación
df_activated_tanda['fecha_afiliacion'] = df_activated_tanda[['onboarding_approved_date', 'terms_accepted_date']].max(axis=1, skipna=False)

# Agregar año de afiliación 
df_activated_tanda['affiliation_year'] = df_activated_tanda.fecha_afiliacion.dt.year

# Agregar cohorte de afiliación (para ello uso la función weeknum() que imita WEEKNUM de Excel)
df_activated_tanda['affiliation_week'] = df_activated_tanda.fecha_afiliacion.apply(weeknum).astype('int')

## 1. Gráfica - Conversion Rate evolution

### Preparación de data

In [11]:
data_cre = df_conversion_rate.copy()

In [13]:
# Agregar columna de días en convertirse
data_cre['dias_en_convertirse'] = (data_cre.fecha_convertido.dt.date - data_cre.fecha_afiliacion.dt.date).dt.days

Hay personas que se convirtieron antes de pasar onboarding. Para esos casos, tomemos la menor fecha entre términos y condiciones y aprobar el onboarding como la fecha en que se afiliaron. 

In [14]:
# Sustituir fecha de afiliación con términos y condiciones
data_cre.loc[data_cre['dias_en_convertirse'] < 0, 'fecha_afiliacion'] = data_cre.loc[data_cre['dias_en_convertirse'] < 0, ['onboarding_approved_date', 'terms_accepted_date']].min(axis=1)

# Sustituir affiliation year y week
data_cre.loc[data_cre['dias_en_convertirse'] < 0, 'affiliation_year'] = data_cre.loc[data_cre['dias_en_convertirse'] < 0, 'fecha_afiliacion'].dt.year 
data_cre.loc[data_cre['dias_en_convertirse'] < 0, 'affiliation_week'] = data_cre.loc[data_cre['dias_en_convertirse'] < 0, 'fecha_afiliacion'].apply(weeknum).astype('int')

# Sustituir dias en convertirse
data_cre.loc[data_cre['dias_en_convertirse'] < 0, 'dias_en_convertirse'] = (data_cre.loc[data_cre['dias_en_convertirse'] < 0, 'fecha_convertido'].dt.date - data_cre.loc[data_cre['dias_en_convertirse'] < 0, 'fecha_afiliacion'].dt.date).dt.days

In [ ]:
data_cre[data_cre['dias_en_convertirse'] < 0]

Calculemos la agrupación por días en convertirse, según el cohorte de afiliación. 

In [16]:
# Crear rango para mis bins
rango_bins_dias_en_convertirse = np.arange(0, 36, 1)
rango_bins_dias_en_convertirse = np.append(rango_bins_dias_en_convertirse, 10000)
rango_bins_dias_en_convertirse

array([    0,     1,     2,     3,     4,     5,     6,     7,     8,
           9,    10,    11,    12,    13,    14,    15,    16,    17,
          18,    19,    20,    21,    22,    23,    24,    25,    26,
          27,    28,    29,    30,    31,    32,    33,    34,    35,
       10000])

In [17]:
# Crear labels para mis bins
labels_rango_bins_dias_en_convertirse = [str(i) for i in np.arange(0, 36, 1)]
print(labels_rango_bins_dias_en_convertirse)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35']


In [18]:
data_cohortes = data_cre.groupby(['affiliation_year', 'affiliation_week', 
                  pd.cut(data_cre["dias_en_convertirse"], 
                         bins=rango_bins_dias_en_convertirse, 
                         right=False, 
                         labels=labels_rango_bins_dias_en_convertirse)])['usuario_id'].count().reset_index()

In [19]:
# Agregar semanas que no tenían ningún registro; así, quedarán con un cero. 
labels = labels_rango_bins_dias_en_convertirse
id_o_user_id = 'usuario_id'
dian_en = "dias_en_convertirse"

for year in [2021, 2022]:
    for week in list(range(2, 54)):
    
    # Checar si existe esa semana; si no existe, créala
        if ((data_cohortes['affiliation_year'] == year) & (data_cohortes['affiliation_week'] == week)).any() == False:
            #Crear año y semana con valores en cero
            d = {'affiliation_year': [year for _ in labels], 'affiliation_week': [week for _ in labels], dian_en: [label for label in labels], id_o_user_id: [0 for _ in labels]}
            df_data = pd.DataFrame(data=d)

            data_cohortes = pd.concat([data_cohortes, df_data])

Eliminemos las semanas vacías que pandas agrega automáticamente

In [20]:
# Sacar año y semana actuales
año_actual = datetime.today().year
semana_actual = weeknum(datetime.today())

# Eliminar signup_weeks agregadas automáticamente por pandas
data_cohortes = data_cohortes[~((data_cohortes.affiliation_year == 2021) & (data_cohortes.affiliation_week < 44)) 
                                & ~((data_cohortes.affiliation_year == año_actual) & (data_cohortes.affiliation_week > semana_actual))].reset_index(drop=True)

In [21]:
data_cohortes

,affiliation_year,affiliation_week,dias_en_convertirse,usuario_id
0,2021,44,0,0
1,2021,44,1,1
2,2021,44,2,0
3,2021,44,3,0
4,2021,44,4,0
...,...,...,...,...
1039,2022,19,31,0
1040,2022,19,32,0
1041,2022,19,33,0
1042,2022,19,34,0


Eliminemos los cohortes en los cuales tenemos cero convertidos

In [22]:
data_cohortes_sin_data = data_cre.groupby(['affiliation_year', 'affiliation_week'], as_index=False)['dias_en_convertirse'].count()
data_cohortes_sin_data.rename({'dias_en_convertirse': 'convertidos'}, axis=1, inplace=True)

zip_sin_data = data_cohortes_sin_data.loc[data_cohortes_sin_data.convertidos == 0, ['affiliation_year', 'affiliation_week']]
lista_cohortes_sin_data = list(zip(zip_sin_data.affiliation_year, zip_sin_data.affiliation_week))
print(lista_cohortes_sin_data)

[(2022, 1)]


In [23]:
for cohorte in lista_cohortes_sin_data:
    data_cohortes = data_cohortes.loc[~((data_cohortes.affiliation_year == cohorte[0]) & (data_cohortes.affiliation_week == cohorte[1]))]

data_cohortes.affiliation_week.unique()

array([44, 45, 46, 47, 48, 49, 50, 51, 52, 53,  2,  3,  4,  5,  6,  7,  8,
        9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], dtype=int64)

Saquemos los días en convertirse acumulados

In [24]:
data_cohortes['dias_en_convertirse_cumulative'] = data_cohortes.groupby(['affiliation_year', 'affiliation_week'])['usuario_id'].cumsum()

C:\Users\franpc\AppData\Local\Temp/ipykernel_20568/2754537105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cohortes['dias_en_convertirse_cumulative'] = data_cohortes.groupby(['affiliation_year', 'affiliation_week'])['usuario_id'].cumsum()


In [25]:
data_cohortes

,affiliation_year,affiliation_week,dias_en_convertirse,usuario_id,dias_en_convertirse_cumulative
0,2021,44,0,0,0
1,2021,44,1,1,1
2,2021,44,2,0,1
3,2021,44,3,0,1
4,2021,44,4,0,1
...,...,...,...,...,...
1039,2022,19,31,0,2
1040,2022,19,32,0,2
1041,2022,19,33,0,2
1042,2022,19,34,0,2


Agreguemos columna con los afiliados totales por cohorte, para después sacar el porcentaje de conversión por cohorte de afiliación.

In [26]:
# Tabla con afiliados totales por cohorte
afiliados_total_por_cohorte = data_cre[data_cre.fecha_afiliacion.notnull()].groupby(['affiliation_year', 'affiliation_week'], as_index=False)['usuario_id'].size()
afiliados_total_por_cohorte.rename({'size': 'total_afiliados_cohorte'}, axis=1, inplace=True)
afiliados_total_por_cohorte.head(6)

,affiliation_year,affiliation_week,total_afiliados_cohorte
0,2021,44,9
1,2021,45,51
2,2021,46,43
3,2021,47,19
4,2021,48,63
5,2021,49,2


In [27]:
# Unir con la tabla general de data_cohortes
data_cohortes_per = data_cohortes.merge(afiliados_total_por_cohorte[['affiliation_year', 'affiliation_week', 'total_afiliados_cohorte']], how='left', left_on=['affiliation_year','affiliation_week'], right_on=['affiliation_year','affiliation_week'])
data_cohortes_per.head()

,affiliation_year,affiliation_week,dias_en_convertirse,usuario_id,dias_en_convertirse_cumulative,total_afiliados_cohorte
0,2021,44,0,0,0,9
1,2021,44,1,1,1,9
2,2021,44,2,0,1,9
3,2021,44,3,0,1,9
4,2021,44,4,0,1,9


In [28]:
data_cohortes_per['dias_en_convertirse_total_percentage'] = (data_cohortes_per['dias_en_convertirse_cumulative'] / data_cohortes_per['total_afiliados_cohorte'] * 100).round(1)
data_cohortes_per

,affiliation_year,affiliation_week,dias_en_convertirse,usuario_id,dias_en_convertirse_cumulative,total_afiliados_cohorte,dias_en_convertirse_total_percentage
0,2021,44,0,0,0,9,0.0
1,2021,44,1,1,1,9,11.1
2,2021,44,2,0,1,9,11.1
3,2021,44,3,0,1,9,11.1
4,2021,44,4,0,1,9,11.1
...,...,...,...,...,...,...,...
1003,2022,19,31,0,2,7,28.6
1004,2022,19,32,0,2,7,28.6
1005,2022,19,33,0,2,7,28.6
1006,2022,19,34,0,2,7,28.6


Data final:

In [29]:
data_cohortes_per.head()

,affiliation_year,affiliation_week,dias_en_convertirse,usuario_id,dias_en_convertirse_cumulative,total_afiliados_cohorte,dias_en_convertirse_total_percentage
0,2021,44,0,0,0,9,0.0
1,2021,44,1,1,1,9,11.1
2,2021,44,2,0,1,9,11.1
3,2021,44,3,0,1,9,11.1
4,2021,44,4,0,1,9,11.1


### Transformación de data para graficar

In [30]:
cohort_pivot = pd.pivot_table(data_cohortes_per, values='dias_en_convertirse_total_percentage', index='dias_en_convertirse', columns=['affiliation_year', 'affiliation_week'])
cohort_pivot.columns = cohort_pivot.columns.map(lambda x: '|'.join([str(i) for i in x]))
cohort_pivot.reset_index(inplace=True)
cohort_pivot

,dias_en_convertirse,2021|44,2021|45,2021|46,2021|47,2021|48,2021|49,2021|50,2021|51,2021|52,2021|53,2022|2,2022|3,2022|4,2022|5,2022|6,2022|7,2022|8,2022|9,2022|10,2022|11,2022|12,2022|13,2022|14,2022|15,2022|16,2022|17,2022|18,2022|19
0,0,0.0,2.0,0.0,0.0,1.6,0.0,0.0,4.7,2.3,1.5,8.6,7.1,2.4,1.3,2.3,1.6,1.5,0.0,0.0,0.0,0.0,0.0,4.2,0.0,2.3,4.8,2.9,14.3
1,1,11.1,5.9,0.0,5.3,1.6,50.0,0.0,9.3,18.2,27.7,39.7,29.6,25.6,32.1,27.5,28.6,36.8,3.4,0.0,0.0,4.3,5.4,12.5,2.4,4.5,4.8,2.9,28.6
2,10,44.4,11.8,16.3,21.1,1.6,50.0,0.0,27.9,34.1,35.4,56.9,42.9,36.6,48.7,33.6,42.9,50.0,16.9,17.8,12.0,13.0,14.3,20.8,9.8,9.1,11.9,8.6,28.6
3,11,55.6,11.8,16.3,21.1,1.6,50.0,0.0,30.2,34.1,36.9,56.9,42.9,37.8,48.7,33.6,42.9,50.0,16.9,17.8,14.0,13.0,14.3,20.8,9.8,9.1,11.9,8.6,28.6
4,12,55.6,11.8,16.3,21.1,1.6,50.0,3.6,32.6,36.4,36.9,56.9,42.9,37.8,48.7,33.6,42.9,50.0,16.9,17.8,14.0,15.2,14.3,20.8,9.8,9.1,11.9,8.6,28.6
5,13,55.6,11.8,16.3,21.1,3.2,50.0,7.1,32.6,36.4,36.9,56.9,42.9,37.8,48.7,33.6,42.9,50.0,16.9,17.8,14.0,15.2,14.3,20.8,9.8,9.1,11.9,8.6,28.6
6,14,55.6,11.8,16.3,21.1,3.2,50.0,7.1,32.6,38.6,38.5,56.9,42.9,39.0,50.0,33.6,42.9,50.0,16.9,17.8,14.0,15.2,14.3,20.8,12.2,9.1,11.9,8.6,28.6
7,15,55.6,13.7,16.3,21.1,3.2,50.0,7.1,32.6,40.9,41.5,56.9,44.9,39.0,50.0,33.6,42.9,50.0,16.9,17.8,16.0,15.2,14.3,20.8,12.2,11.4,11.9,8.6,28.6
8,16,55.6,13.7,16.3,21.1,3.2,50.0,7.1,32.6,40.9,41.5,56.9,44.9,39.0,50.0,33.6,42.9,50.0,16.9,17.8,18.0,17.4,14.3,20.8,12.2,11.4,11.9,8.6,28.6
9,17,55.6,13.7,16.3,21.1,3.2,50.0,7.1,32.6,40.9,41.5,58.6,45.9,39.0,50.0,33.6,42.9,50.0,16.9,17.8,18.0,17.4,14.3,20.8,12.2,11.4,11.9,8.6,28.6


In [31]:
# Cambiar columna de días a numérica para poder ordenarla
cohort_pivot = cohort_pivot.astype({'dias_en_convertirse': 'float'})

# Ordenar valores de manera correcta
cohort_pivot = cohort_pivot.sort_values(by='dias_en_convertirse').reset_index(drop=True)
cohort_pivot

,dias_en_convertirse,2021|44,2021|45,2021|46,2021|47,2021|48,2021|49,2021|50,2021|51,2021|52,2021|53,2022|2,2022|3,2022|4,2022|5,2022|6,2022|7,2022|8,2022|9,2022|10,2022|11,2022|12,2022|13,2022|14,2022|15,2022|16,2022|17,2022|18,2022|19
0,0.0,0.0,2.0,0.0,0.0,1.6,0.0,0.0,4.7,2.3,1.5,8.6,7.1,2.4,1.3,2.3,1.6,1.5,0.0,0.0,0.0,0.0,0.0,4.2,0.0,2.3,4.8,2.9,14.3
1,1.0,11.1,5.9,0.0,5.3,1.6,50.0,0.0,9.3,18.2,27.7,39.7,29.6,25.6,32.1,27.5,28.6,36.8,3.4,0.0,0.0,4.3,5.4,12.5,2.4,4.5,4.8,2.9,28.6
2,2.0,11.1,5.9,7.0,5.3,1.6,50.0,0.0,14.0,25.0,30.8,44.8,34.7,29.3,32.1,27.5,31.7,38.2,5.1,4.4,4.0,4.3,5.4,16.7,2.4,6.8,4.8,2.9,28.6
3,3.0,11.1,7.8,11.6,5.3,1.6,50.0,0.0,23.3,31.8,33.8,53.4,35.7,31.7,35.9,28.2,33.3,44.1,5.1,8.9,4.0,6.5,7.1,18.8,2.4,6.8,4.8,2.9,28.6
4,4.0,11.1,9.8,11.6,15.8,1.6,50.0,0.0,23.3,34.1,33.8,55.2,37.8,32.9,39.7,29.8,33.3,45.6,13.6,11.1,4.0,6.5,7.1,20.8,4.9,6.8,4.8,5.7,28.6
5,5.0,11.1,9.8,11.6,15.8,1.6,50.0,0.0,25.6,34.1,35.4,55.2,38.8,32.9,42.3,29.8,36.5,47.1,13.6,11.1,4.0,10.9,8.9,20.8,4.9,6.8,4.8,8.6,28.6
6,6.0,33.3,9.8,14.0,15.8,1.6,50.0,0.0,25.6,34.1,35.4,56.9,41.8,32.9,46.2,32.1,36.5,48.5,13.6,11.1,6.0,10.9,10.7,20.8,4.9,6.8,4.8,8.6,28.6
7,7.0,33.3,9.8,14.0,21.1,1.6,50.0,0.0,25.6,34.1,35.4,56.9,41.8,35.4,47.4,32.1,38.1,48.5,16.9,11.1,8.0,10.9,10.7,20.8,4.9,9.1,11.9,8.6,28.6
8,8.0,44.4,9.8,14.0,21.1,1.6,50.0,0.0,25.6,34.1,35.4,56.9,42.9,35.4,48.7,32.8,41.3,50.0,16.9,11.1,10.0,10.9,10.7,20.8,7.3,9.1,11.9,8.6,28.6
9,9.0,44.4,9.8,16.3,21.1,1.6,50.0,0.0,25.6,34.1,35.4,56.9,42.9,36.6,48.7,32.8,41.3,50.0,16.9,17.8,10.0,10.9,14.3,20.8,7.3,9.1,11.9,8.6,28.6


In [32]:
# Eliminar los días que todavía no suceden
for i in range((len(cohort_pivot) // 7)+1): 
    i += 1
    cohort_pivot.iloc[7*(i-1):, -i] = ""

In [33]:
cohort_pivot

,dias_en_convertirse,2021|44,2021|45,2021|46,2021|47,2021|48,2021|49,2021|50,2021|51,2021|52,2021|53,2022|2,2022|3,2022|4,2022|5,2022|6,2022|7,2022|8,2022|9,2022|10,2022|11,2022|12,2022|13,2022|14,2022|15,2022|16,2022|17,2022|18,2022|19
0,0.0,0.0,2.0,0.0,0.0,1.6,0.0,0.0,4.7,2.3,1.5,8.6,7.1,2.4,1.3,2.3,1.6,1.5,0.0,0.0,0.0,0.0,0.0,4.2,0.0,2.3,4.8,2.9,
1,1.0,11.1,5.9,0.0,5.3,1.6,50.0,0.0,9.3,18.2,27.7,39.7,29.6,25.6,32.1,27.5,28.6,36.8,3.4,0.0,0.0,4.3,5.4,12.5,2.4,4.5,4.8,2.9,
2,2.0,11.1,5.9,7.0,5.3,1.6,50.0,0.0,14.0,25.0,30.8,44.8,34.7,29.3,32.1,27.5,31.7,38.2,5.1,4.4,4.0,4.3,5.4,16.7,2.4,6.8,4.8,2.9,
3,3.0,11.1,7.8,11.6,5.3,1.6,50.0,0.0,23.3,31.8,33.8,53.4,35.7,31.7,35.9,28.2,33.3,44.1,5.1,8.9,4.0,6.5,7.1,18.8,2.4,6.8,4.8,2.9,
4,4.0,11.1,9.8,11.6,15.8,1.6,50.0,0.0,23.3,34.1,33.8,55.2,37.8,32.9,39.7,29.8,33.3,45.6,13.6,11.1,4.0,6.5,7.1,20.8,4.9,6.8,4.8,5.7,
5,5.0,11.1,9.8,11.6,15.8,1.6,50.0,0.0,25.6,34.1,35.4,55.2,38.8,32.9,42.3,29.8,36.5,47.1,13.6,11.1,4.0,10.9,8.9,20.8,4.9,6.8,4.8,8.6,
6,6.0,33.3,9.8,14.0,15.8,1.6,50.0,0.0,25.6,34.1,35.4,56.9,41.8,32.9,46.2,32.1,36.5,48.5,13.6,11.1,6.0,10.9,10.7,20.8,4.9,6.8,4.8,8.6,
7,7.0,33.3,9.8,14.0,21.1,1.6,50.0,0.0,25.6,34.1,35.4,56.9,41.8,35.4,47.4,32.1,38.1,48.5,16.9,11.1,8.0,10.9,10.7,20.8,4.9,9.1,11.9,,
8,8.0,44.4,9.8,14.0,21.1,1.6,50.0,0.0,25.6,34.1,35.4,56.9,42.9,35.4,48.7,32.8,41.3,50.0,16.9,11.1,10.0,10.9,10.7,20.8,7.3,9.1,11.9,,
9,9.0,44.4,9.8,16.3,21.1,1.6,50.0,0.0,25.6,34.1,35.4,56.9,42.9,36.6,48.7,32.8,41.3,50.0,16.9,17.8,10.0,10.9,14.3,20.8,7.3,9.1,11.9,,


### Pasar a G Sheets

In [34]:
sheet = client.open("Business_Health_Dashboard")  #open sheet

#replace sheet_name with the name that corresponds to yours, e.g, it can be sheet1
sheet = sheet.worksheet("data_conversion_rate")

In [35]:
sheet.update([cohort_pivot.columns.values.tolist()] + cohort_pivot.values.tolist())

{'spreadsheetId': '1ZKpgLgKOSt-8JuokkfHOcTBAG72ER73-RcE-g7sttJ4',
 'updatedRange': 'data_conversion_rate!A1:AC37',
 'updatedRows': 37,
 'updatedColumns': 29,
 'updatedCells': 1073}

## 2. Gráfica - Activated before converting

### Preparación de data

In [36]:
data_abc = df_activated_tanda.copy()

In [ ]:
data_abc.head()

Las dos condiciones que busco para una tanda activada por usuario son estas:
1. Que el usuario se haya afiliado antes de cancelar (basta con comparar que la fecha de afiliación sea menos a la de cancelación).
2. Que se haya creado una tanda, que el usuario se haya afiliado y que no se haya cancelado (o sea, que el campo de cancelación sea nulo). En este caso, el dataframe muestra solo tandas creadas y de usuarios afiliados (o sea que ya está filtrado por dos condiciones). Para la última, solo checo que el campo de cancelación esté nulo.  

In [38]:
# Filtrar según las dos condiciones anteriores
data_abc = data_abc[(data_abc.fecha_afiliacion < data_abc.fecha_cancelacion_tanda) | (data_abc.fecha_cancelacion_tanda.isnull())]

# Eliminar usuarios duplicados (es decir, que ya han activado varias tandas
data_abc = data_abc.drop_duplicates('id', keep='first')

# Crear columna con fecha de activación de la tanda más joven
data_abc['fecha_activacion_primera_tanda'] = data_abc.fecha_creacion_tanda

# Seleccionar solo a aquellos que activaron una tanda antes de convertirse
data_abc = data_abc[data_abc['fecha_activacion_primera_tanda'] < data_abc['fecha_convertido']]

In [ ]:
data_abc.head()

In [40]:
# Sacar los días que se tardaron en activar una tanda (puede ser negativo)
data_abc['dias_en_activar_tanda'] = (data_abc.fecha_activacion_primera_tanda.dt.date - data_abc.fecha_afiliacion.dt.date).dt.days

In [41]:
# Crear rango para mis bins
rango_bins_dias_en_activar = np.arange(-49, 51, 1)
rango_bins_dias_en_activar = np.insert(rango_bins_dias_en_activar, 0, -10000)
rango_bins_dias_en_activar = np.append(rango_bins_dias_en_activar, 10000)
rango_bins_dias_en_activar

array([-10000,    -49,    -48,    -47,    -46,    -45,    -44,    -43,
          -42,    -41,    -40,    -39,    -38,    -37,    -36,    -35,
          -34,    -33,    -32,    -31,    -30,    -29,    -28,    -27,
          -26,    -25,    -24,    -23,    -22,    -21,    -20,    -19,
          -18,    -17,    -16,    -15,    -14,    -13,    -12,    -11,
          -10,     -9,     -8,     -7,     -6,     -5,     -4,     -3,
           -2,     -1,      0,      1,      2,      3,      4,      5,
            6,      7,      8,      9,     10,     11,     12,     13,
           14,     15,     16,     17,     18,     19,     20,     21,
           22,     23,     24,     25,     26,     27,     28,     29,
           30,     31,     32,     33,     34,     35,     36,     37,
           38,     39,     40,     41,     42,     43,     44,     45,
           46,     47,     48,     49,     50,  10000])

In [42]:
# Crear labels para mis bins
labels_rango_bins_dias_en_activar = [str(i) for i in np.arange(-50, 51, 1)]
print(labels_rango_bins_dias_en_activar)

['-50', '-49', '-48', '-47', '-46', '-45', '-44', '-43', '-42', '-41', '-40', '-39', '-38', '-37', '-36', '-35', '-34', '-33', '-32', '-31', '-30', '-29', '-28', '-27', '-26', '-25', '-24', '-23', '-22', '-21', '-20', '-19', '-18', '-17', '-16', '-15', '-14', '-13', '-12', '-11', '-10', '-9', '-8', '-7', '-6', '-5', '-4', '-3', '-2', '-1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50']


In [43]:
data_cohortes = data_abc.groupby(['affiliation_year', 'affiliation_week', 
                  pd.cut(data_abc["dias_en_activar_tanda"], 
                         bins=rango_bins_dias_en_activar, 
                         right=False, 
                         labels=labels_rango_bins_dias_en_activar)])['id'].count().reset_index()

In [44]:
# Agregar semanas que no tenían ningún registro; así, quedarán con un cero. 
labels = labels_rango_bins_dias_en_activar
id_o_user_id = 'id'
dian_en = "dias_en_activar_tanda"

for year in [2021, 2022]:
    for week in list(range(2, 54)):
    
    # Checar si existe esa semana; si no existe, créala
        if ((data_cohortes['affiliation_year'] == year) & (data_cohortes['affiliation_week'] == week)).any() == False:
            #Crear año y semana con valores en cero
            d = {'affiliation_year': [year for _ in labels], 'affiliation_week': [week for _ in labels], dian_en: [label for label in labels], id_o_user_id: [0 for _ in labels]}
            df_data = pd.DataFrame(data=d)

            data_cohortes = pd.concat([data_cohortes, df_data])

In [45]:
# Eliminar semanas vacías que pandas agrega automáticamente
# Sacar año y semana actuales
año_actual = datetime.today().year
semana_actual = weeknum(datetime.today())

# Eliminar signup_weeks agregadas automáticamente por pandas
data_cohortes = data_cohortes[~((data_cohortes.affiliation_year == 2021) & (data_cohortes.affiliation_week < 44)) 
                                & ~((data_cohortes.affiliation_year == año_actual) & (data_cohortes.affiliation_week > semana_actual))].reset_index(drop=True)

In [46]:
# Agreguemos columna con los afiliados totales por cohorte, para después sacar el porcentaje de activación de tanda por cohorte de afiliación.
data_cohortes['dias_en_activar_tanda_cumulative'] = data_cohortes.groupby(['affiliation_year', 'affiliation_week'])['id'].cumsum()

In [47]:
# Tabla con afiliados totales por cohorte
afiliados_total_por_cohorte = data_cre[data_cre.fecha_afiliacion.notnull()].groupby(['affiliation_year', 'affiliation_week'], as_index=False)['usuario_id'].size()
afiliados_total_por_cohorte.rename({'size': 'total_afiliados_cohorte'}, axis=1, inplace=True)
afiliados_total_por_cohorte.head(6)

,affiliation_year,affiliation_week,total_afiliados_cohorte
0,2021,44,9
1,2021,45,51
2,2021,46,43
3,2021,47,19
4,2021,48,63
5,2021,49,2


In [48]:
# Unir con la tabla general de data_cohortes
data_cohortes_per = data_cohortes.merge(afiliados_total_por_cohorte[['affiliation_year', 'affiliation_week', 'total_afiliados_cohorte']], how='left', left_on=['affiliation_year','affiliation_week'], right_on=['affiliation_year','affiliation_week'])
data_cohortes_per.head()

,affiliation_year,affiliation_week,dias_en_activar_tanda,id,dias_en_activar_tanda_cumulative,total_afiliados_cohorte
0,2021,45,-50,0,0,51
1,2021,45,-49,0,0,51
2,2021,45,-48,0,0,51
3,2021,45,-47,0,0,51
4,2021,45,-46,0,0,51


In [49]:
data_cohortes_per['dias_en_activar_tanda_percentage'] = (data_cohortes_per['dias_en_activar_tanda_cumulative'] / data_cohortes_per['total_afiliados_cohorte'] * 100).round(1)
data_cohortes_per

,affiliation_year,affiliation_week,dias_en_activar_tanda,id,dias_en_activar_tanda_cumulative,total_afiliados_cohorte,dias_en_activar_tanda_percentage
0,2021,45,-50,0,0,51,0.0
1,2021,45,-49,0,0,51,0.0
2,2021,45,-48,0,0,51,0.0
3,2021,45,-47,0,0,51,0.0
4,2021,45,-46,0,0,51,0.0
...,...,...,...,...,...,...,...
2823,2021,49,46,0,0,2,0.0
2824,2021,49,47,0,0,2,0.0
2825,2021,49,48,0,0,2,0.0
2826,2021,49,49,0,0,2,0.0


Data final:

In [50]:
data_cohortes_per

,affiliation_year,affiliation_week,dias_en_activar_tanda,id,dias_en_activar_tanda_cumulative,total_afiliados_cohorte,dias_en_activar_tanda_percentage
0,2021,45,-50,0,0,51,0.0
1,2021,45,-49,0,0,51,0.0
2,2021,45,-48,0,0,51,0.0
3,2021,45,-47,0,0,51,0.0
4,2021,45,-46,0,0,51,0.0
...,...,...,...,...,...,...,...
2823,2021,49,46,0,0,2,0.0
2824,2021,49,47,0,0,2,0.0
2825,2021,49,48,0,0,2,0.0
2826,2021,49,49,0,0,2,0.0


### Transformar data para graficar

In [51]:
cohort_pivot = pd.pivot_table(data_cohortes_per, values='dias_en_activar_tanda_percentage', index='dias_en_activar_tanda', columns=['affiliation_year', 'affiliation_week'])
cohort_pivot.columns = cohort_pivot.columns.map(lambda x: '|'.join([str(i) for i in x]))
cohort_pivot.reset_index(inplace=True)
cohort_pivot

,dias_en_activar_tanda,2021|44,2021|45,2021|46,2021|47,2021|48,2021|49,2021|50,2021|51,2021|52,2021|53,2022|2,2022|3,2022|4,2022|5,2022|6,2022|7,2022|8,2022|9,2022|10,2022|11,2022|12,2022|13,2022|14,2022|15,2022|16,2022|17,2022|18,2022|19
0,-1,0.0,7.8,11.6,15.8,17.5,0.0,10.7,20.9,34.1,30.8,27.6,27.6,20.7,41.0,30.5,34.9,42.6,23.7,6.7,10.0,15.2,10.7,22.9,9.8,9.1,11.9,8.6,14.3
1,-10,0.0,0.0,2.3,5.3,17.5,0.0,10.7,2.3,4.5,7.7,5.2,2.0,4.9,5.1,10.7,11.1,7.4,8.5,4.4,4.0,6.5,3.6,4.2,2.4,0.0,11.9,8.6,14.3
2,-11,0.0,0.0,2.3,5.3,15.9,0.0,10.7,2.3,4.5,7.7,5.2,2.0,4.9,5.1,8.4,9.5,7.4,8.5,4.4,4.0,6.5,3.6,4.2,2.4,0.0,11.9,8.6,14.3
3,-12,0.0,0.0,2.3,5.3,15.9,0.0,10.7,2.3,4.5,7.7,3.4,2.0,4.9,5.1,8.4,9.5,7.4,8.5,4.4,4.0,6.5,3.6,4.2,2.4,0.0,11.9,8.6,14.3
4,-13,0.0,0.0,2.3,0.0,14.3,0.0,10.7,2.3,4.5,7.7,3.4,1.0,4.9,5.1,7.6,9.5,4.4,8.5,4.4,4.0,6.5,3.6,4.2,2.4,0.0,11.9,8.6,14.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,50,0.0,11.8,11.6,15.8,22.2,0.0,14.3,30.2,43.2,40.0,48.3,42.9,37.8,47.4,38.2,41.3,44.1,28.8,15.6,18.0,17.4,16.1,22.9,12.2,11.4,11.9,8.6,14.3
97,6,0.0,11.8,11.6,15.8,17.5,0.0,14.3,30.2,43.2,40.0,48.3,42.9,37.8,46.2,38.2,41.3,44.1,28.8,15.6,18.0,17.4,16.1,22.9,12.2,11.4,11.9,8.6,14.3
98,7,0.0,11.8,11.6,15.8,17.5,0.0,14.3,30.2,43.2,40.0,48.3,42.9,37.8,46.2,38.2,41.3,44.1,28.8,15.6,18.0,17.4,16.1,22.9,12.2,11.4,11.9,8.6,14.3
99,8,0.0,11.8,11.6,15.8,17.5,0.0,14.3,30.2,43.2,40.0,48.3,42.9,37.8,46.2,38.2,41.3,44.1,28.8,15.6,18.0,17.4,16.1,22.9,12.2,11.4,11.9,8.6,14.3


In [52]:
# Cambiar columna de días a numérica para poder ordenarla
cohort_pivot = cohort_pivot.astype({'dias_en_activar_tanda': 'float'})

# Ordenar valores de manera correcta
cohort_pivot = cohort_pivot.sort_values(by='dias_en_activar_tanda').reset_index(drop=True)
cohort_pivot

,dias_en_activar_tanda,2021|44,2021|45,2021|46,2021|47,2021|48,2021|49,2021|50,2021|51,2021|52,2021|53,2022|2,2022|3,2022|4,2022|5,2022|6,2022|7,2022|8,2022|9,2022|10,2022|11,2022|12,2022|13,2022|14,2022|15,2022|16,2022|17,2022|18,2022|19
0,-50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.5,0.0,0.0,0.0,0.0,0.8,1.6,1.5,1.7,2.2,2.0,4.3,3.6,2.1,0.0,0.0,7.1,8.6,14.3
1,-49.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.5,0.0,0.0,0.0,0.0,0.8,1.6,1.5,1.7,2.2,2.0,4.3,3.6,2.1,0.0,0.0,7.1,8.6,14.3
2,-48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.5,0.0,0.0,0.0,0.0,0.8,1.6,1.5,1.7,2.2,2.0,4.3,3.6,2.1,0.0,0.0,7.1,8.6,14.3
3,-47.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,3.1,0.0,0.0,0.0,0.0,0.8,1.6,1.5,3.4,2.2,2.0,4.3,3.6,2.1,0.0,0.0,7.1,8.6,14.3
4,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,3.1,0.0,0.0,0.0,0.0,0.8,1.6,1.5,3.4,4.4,2.0,4.3,3.6,2.1,0.0,0.0,7.1,8.6,14.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,46.0,0.0,11.8,11.6,15.8,20.6,0.0,14.3,30.2,43.2,40.0,48.3,42.9,37.8,47.4,38.2,41.3,44.1,28.8,15.6,18.0,17.4,16.1,22.9,12.2,11.4,11.9,8.6,14.3
97,47.0,0.0,11.8,11.6,15.8,20.6,0.0,14.3,30.2,43.2,40.0,48.3,42.9,37.8,47.4,38.2,41.3,44.1,28.8,15.6,18.0,17.4,16.1,22.9,12.2,11.4,11.9,8.6,14.3
98,48.0,0.0,11.8,11.6,15.8,20.6,0.0,14.3,30.2,43.2,40.0,48.3,42.9,37.8,47.4,38.2,41.3,44.1,28.8,15.6,18.0,17.4,16.1,22.9,12.2,11.4,11.9,8.6,14.3
99,49.0,0.0,11.8,11.6,15.8,20.6,0.0,14.3,30.2,43.2,40.0,48.3,42.9,37.8,47.4,38.2,41.3,44.1,28.8,15.6,18.0,17.4,16.1,22.9,12.2,11.4,11.9,8.6,14.3


In [53]:
# Eliminar los días que todavía no suceden
for i in range(( (len(cohort_pivot)-51) // 7)+1): 
    i += 1
    cohort_pivot.iloc[50 + (7*(i-1)):, -i] = ""

In [ ]:
cohort_pivot

### Pasar a G Sheets

In [54]:
sheet = client.open("Business_Health_Dashboard")  #open sheet

#replace sheet_name with the name that corresponds to yours, e.g, it can be sheet1
sheet = sheet.worksheet("data_activation_rate")

In [55]:
sheet.update([cohort_pivot.columns.values.tolist()] + cohort_pivot.values.tolist())

{'spreadsheetId': '1ZKpgLgKOSt-8JuokkfHOcTBAG72ER73-RcE-g7sttJ4',
 'updatedRange': 'data_activation_rate!A1:AC102',
 'updatedRows': 102,
 'updatedColumns': 29,
 'updatedCells': 2958}

## 3. Gráfica - Cancelled before converting

Puesto que cada usuario activó y canceló varias tandas, puedo sacar el promedio de días que se tardan en cancelar. 

### Preparación de data

In [56]:
data_cr = df_activated_tanda.copy()

In [57]:
# Filtrar según tanda activada
data_cr = data_cr[(data_cr.fecha_afiliacion < data_cr.fecha_cancelacion_tanda) | (data_cr.fecha_cancelacion_tanda.isnull())]
print(data_cr.shape)

# Considerar solo las tandas que se han cancelado (de lo contrario, no puedo calcular cuánto se tardan en cancelar)
data_cr = data_cr[data_cr.fecha_cancelacion_tanda.notnull()]
print(data_cr.shape)

# Solo considerar las cancelaciones que se dieron antes de que se conviertieran
data_cr = data_cr[data_cr.fecha_cancelacion_tanda < data_cr.fecha_convertido]
print(data_cr.shape)

(1858, 11)
(1506, 11)
(51, 11)


In [58]:
# Crear columna con días en que se tardaron en cancelar la tanda
data_cr['dias_en_cancelar'] = (data_cr.fecha_cancelacion_tanda - data_cr.fecha_creacion_tanda).dt.days

In [59]:
# Crear rango para mis bins
rango_bins_dias_en_cancelar = np.arange(0, 31, 1)
rango_bins_dias_en_cancelar = np.append(rango_bins_dias_en_cancelar, 10000)
rango_bins_dias_en_cancelar

array([    0,     1,     2,     3,     4,     5,     6,     7,     8,
           9,    10,    11,    12,    13,    14,    15,    16,    17,
          18,    19,    20,    21,    22,    23,    24,    25,    26,
          27,    28,    29,    30, 10000])

In [60]:
# Crear labels para mis bins
labels_rango_bins_dias_en_cancelar = [str(i) for i in np.arange(0, 31, 1)]
print(labels_rango_bins_dias_en_cancelar)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']


In [61]:
data_cohortes = data_cr.groupby(['affiliation_year', 'affiliation_week', 
                  pd.cut(data_cr["dias_en_cancelar"], 
                         bins=rango_bins_dias_en_cancelar, 
                         right=False, 
                         labels=labels_rango_bins_dias_en_cancelar)])['id'].count().reset_index()

In [62]:
# Agregar semanas que no tenían ningún registro; así, quedarán con un cero. 
labels = labels_rango_bins_dias_en_cancelar
id_o_user_id = 'id'
dian_en = "dias_en_cancelar"

for year in [2021, 2022]:
    for week in list(range(2, 54)):
    
    # Checar si existe esa semana; si no existe, créala
        if ((data_cohortes['affiliation_year'] == year) & (data_cohortes['affiliation_week'] == week)).any() == False:
            #Crear año y semana con valores en cero
            d = {'affiliation_year': [year for _ in labels], 'affiliation_week': [week for _ in labels], dian_en: [label for label in labels], id_o_user_id: [0 for _ in labels]}
            df_data = pd.DataFrame(data=d)

            data_cohortes = pd.concat([data_cohortes, df_data])

In [63]:
# Ordenar valores con las filas agregadas
data_cohortes = data_cohortes.sort_values(by=['affiliation_year', 'affiliation_week'])
data_cohortes[data_cohortes.affiliation_year == 2022].affiliation_week.unique()

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53], dtype=int64)

In [64]:
# Eliminar semanas vacías que pandas agrega automáticamente
# Sacar año y semana actuales
año_actual = datetime.today().year
semana_actual = weeknum(datetime.today())

# Eliminar signup_weeks agregadas automáticamente por pandas
data_cohortes = data_cohortes[~((data_cohortes.affiliation_year == 2021) & (data_cohortes.affiliation_week < 44)) 
                                & ~((data_cohortes.affiliation_year == año_actual) & (data_cohortes.affiliation_week > semana_actual))].reset_index(drop=True)

In [65]:
# Agregar columna con cancelaciones cumulativas
data_cohortes['dias_en_cancelar_tanda_cumulative'] = data_cohortes.groupby(['affiliation_year', 'affiliation_week'])['id'].cumsum()

In [66]:
# Obtener número total de tandas activadas por cohorte
activaciones_totales = df_activated_tanda.copy()

activaciones_totales = activaciones_totales[(activaciones_totales.fecha_afiliacion < activaciones_totales.fecha_cancelacion_tanda) | (activaciones_totales.fecha_cancelacion_tanda.isnull())]

# Eliminar usuarios duplicados (es decir, que ya han activado varias tandas
activaciones_totales = activaciones_totales.drop_duplicates('id', keep='first')

# Crear columna con fecha de activación de la tanda más joven
activaciones_totales['fecha_activacion_primera_tanda'] = activaciones_totales.fecha_creacion_tanda

# Seleccionar solo a aquellos que activaron una tanda antes de convertirse
activaciones_totales = activaciones_totales[activaciones_totales['fecha_activacion_primera_tanda'] < activaciones_totales['fecha_convertido']]

# Agrupar por cohorte
activaciones_por_cohorte = activaciones_totales.groupby(['affiliation_year', 'affiliation_week'], as_index=False)['id'].size()
activaciones_por_cohorte.rename({'size': 'activaciones_cohorte'}, axis=1, inplace=True)
activaciones_por_cohorte.head(6)

,affiliation_year,affiliation_week,activaciones_cohorte
0,2021,45,6
1,2021,46,5
2,2021,47,3
3,2021,48,14
4,2021,50,4
5,2021,51,13


In [67]:
# Unir con la tabla general de data_cohortes
data_cohortes_per = data_cohortes.merge(activaciones_por_cohorte[['affiliation_year', 'affiliation_week', 'activaciones_cohorte']], how='left', left_on=['affiliation_year','affiliation_week'], right_on=['affiliation_year','affiliation_week'])
data_cohortes_per.head()

,affiliation_year,affiliation_week,dias_en_cancelar,id,dias_en_cancelar_tanda_cumulative,activaciones_cohorte
0,2021,44,0,0,0,NaN
1,2021,44,1,0,0,NaN
2,2021,44,2,0,0,NaN
3,2021,44,3,0,0,NaN
4,2021,44,4,0,0,NaN


In [68]:
# Obtener el porcentaje de cancelaciones (con relación a activaciones) por cohorte
data_cohortes_per['dias_en_cancelar_tanda_percentage'] = (data_cohortes_per['dias_en_cancelar_tanda_cumulative'] / data_cohortes_per['activaciones_cohorte'] * 100).round(1)
data_cohortes_per

,affiliation_year,affiliation_week,dias_en_cancelar,id,dias_en_cancelar_tanda_cumulative,activaciones_cohorte,dias_en_cancelar_tanda_percentage
0,2021,44,0,0,0,NaN,NaN
1,2021,44,1,0,0,NaN,NaN
2,2021,44,2,0,0,NaN,NaN
3,2021,44,3,0,0,NaN,NaN
4,2021,44,4,0,0,NaN,NaN
...,...,...,...,...,...,...,...
863,2022,19,26,0,0,1.0,0.0
864,2022,19,27,0,0,1.0,0.0
865,2022,19,28,0,0,1.0,0.0
866,2022,19,29,0,0,1.0,0.0


In [69]:
# Rellenar los valores nulos
data_cohortes_per.fillna(0, inplace=True)

Data final:

In [70]:
data_cohortes_per

,affiliation_year,affiliation_week,dias_en_cancelar,id,dias_en_cancelar_tanda_cumulative,activaciones_cohorte,dias_en_cancelar_tanda_percentage
0,2021,44,0,0,0,0.0,0.0
1,2021,44,1,0,0,0.0,0.0
2,2021,44,2,0,0,0.0,0.0
3,2021,44,3,0,0,0.0,0.0
4,2021,44,4,0,0,0.0,0.0
...,...,...,...,...,...,...,...
863,2022,19,26,0,0,1.0,0.0
864,2022,19,27,0,0,1.0,0.0
865,2022,19,28,0,0,1.0,0.0
866,2022,19,29,0,0,1.0,0.0


### Transformar data para graficar

In [71]:
cohort_pivot = pd.pivot_table(data_cohortes_per, values='dias_en_cancelar_tanda_percentage', index='dias_en_cancelar', columns=['affiliation_year', 'affiliation_week'])
cohort_pivot.columns = cohort_pivot.columns.map(lambda x: '|'.join([str(i) for i in x]))
cohort_pivot.reset_index(inplace=True)
cohort_pivot

,dias_en_cancelar,2021|44,2021|45,2021|46,2021|47,2021|48,2021|49,2021|50,2021|51,2021|52,2021|53,2022|2,2022|3,2022|4,2022|5,2022|6,2022|7,2022|8,2022|9,2022|10,2022|11,2022|12,2022|13,2022|14,2022|15,2022|16,2022|17,2022|18,2022|19
0,0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,0.0,3.8,3.6,11.9,3.2,2.7,2.0,0.0,3.3,0.0,0.0,0.0,0.0,11.1,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,5.3,3.8,3.6,16.7,3.2,2.7,2.0,0.0,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,0.0
2,10,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,7.7,3.6,19.0,3.2,2.7,6.0,7.7,6.7,5.9,0.0,0.0,12.5,44.4,0.0,0.0,0.0,0.0,0.0,0.0
3,11,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,7.7,3.6,19.0,3.2,2.7,6.0,7.7,6.7,5.9,0.0,0.0,12.5,44.4,0.0,0.0,0.0,0.0,0.0,0.0
4,12,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,7.7,7.1,19.0,3.2,2.7,6.0,7.7,6.7,5.9,0.0,0.0,12.5,44.4,0.0,0.0,20.0,0.0,0.0,0.0
5,13,0.0,0.0,0.0,0.0,21.4,0.0,25.0,7.7,10.5,7.7,7.1,19.0,3.2,5.4,6.0,7.7,6.7,5.9,0.0,0.0,12.5,44.4,0.0,0.0,20.0,0.0,0.0,0.0
6,14,0.0,0.0,0.0,0.0,21.4,0.0,25.0,7.7,10.5,7.7,7.1,19.0,3.2,5.4,6.0,7.7,6.7,5.9,0.0,0.0,12.5,44.4,0.0,0.0,20.0,0.0,0.0,0.0
7,15,0.0,0.0,0.0,0.0,21.4,0.0,25.0,7.7,10.5,11.5,7.1,19.0,3.2,5.4,6.0,7.7,6.7,5.9,0.0,0.0,12.5,44.4,0.0,0.0,20.0,0.0,0.0,0.0
8,16,0.0,0.0,0.0,0.0,21.4,0.0,25.0,15.4,10.5,11.5,7.1,19.0,3.2,5.4,6.0,7.7,6.7,5.9,0.0,0.0,12.5,44.4,0.0,0.0,20.0,0.0,0.0,0.0
9,17,0.0,0.0,0.0,0.0,21.4,0.0,25.0,15.4,10.5,11.5,7.1,19.0,3.2,5.4,6.0,7.7,10.0,11.8,0.0,0.0,12.5,44.4,0.0,0.0,20.0,0.0,0.0,0.0


In [72]:
# Cambiar columna de días a numérica para poder ordenarla
cohort_pivot = cohort_pivot.astype({'dias_en_cancelar': 'float'})

# Ordenar valores de manera correcta
cohort_pivot = cohort_pivot.sort_values(by='dias_en_cancelar').reset_index(drop=True)
cohort_pivot

,dias_en_cancelar,2021|44,2021|45,2021|46,2021|47,2021|48,2021|49,2021|50,2021|51,2021|52,2021|53,2022|2,2022|3,2022|4,2022|5,2022|6,2022|7,2022|8,2022|9,2022|10,2022|11,2022|12,2022|13,2022|14,2022|15,2022|16,2022|17,2022|18,2022|19
0,0.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,0.0,3.8,3.6,11.9,3.2,2.7,2.0,0.0,3.3,0.0,0.0,0.0,0.0,11.1,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,5.3,3.8,3.6,16.7,3.2,2.7,2.0,0.0,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,5.3,3.8,3.6,19.0,3.2,2.7,4.0,3.8,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,5.3,3.8,3.6,19.0,3.2,2.7,4.0,3.8,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,3.8,3.6,19.0,3.2,2.7,4.0,7.7,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,0.0
5,5.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,3.8,3.6,19.0,3.2,2.7,4.0,7.7,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,0.0
6,6.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,3.8,3.6,19.0,3.2,2.7,6.0,7.7,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,0.0
7,7.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,7.7,3.6,19.0,3.2,2.7,6.0,7.7,6.7,5.9,0.0,0.0,12.5,22.2,0.0,0.0,0.0,0.0,0.0,0.0
8,8.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,7.7,3.6,19.0,3.2,2.7,6.0,7.7,6.7,5.9,0.0,0.0,12.5,33.3,0.0,0.0,0.0,0.0,0.0,0.0
9,9.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,7.7,3.6,19.0,3.2,2.7,6.0,7.7,6.7,5.9,0.0,0.0,12.5,44.4,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
# Eliminar los días que todavía no suceden
for i in range((len(cohort_pivot) // 7)+1): 
    i += 1
    cohort_pivot.iloc[7*(i-1):, -i] = ""

In [74]:
cohort_pivot

,dias_en_cancelar,2021|44,2021|45,2021|46,2021|47,2021|48,2021|49,2021|50,2021|51,2021|52,2021|53,2022|2,2022|3,2022|4,2022|5,2022|6,2022|7,2022|8,2022|9,2022|10,2022|11,2022|12,2022|13,2022|14,2022|15,2022|16,2022|17,2022|18,2022|19
0,0.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,0.0,3.8,3.6,11.9,3.2,2.7,2.0,0.0,3.3,0.0,0.0,0.0,0.0,11.1,0.0,0.0,0.0,0.0,0.0,
1,1.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,5.3,3.8,3.6,16.7,3.2,2.7,2.0,0.0,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,
2,2.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,5.3,3.8,3.6,19.0,3.2,2.7,4.0,3.8,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,
3,3.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,5.3,3.8,3.6,19.0,3.2,2.7,4.0,3.8,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,
4,4.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,3.8,3.6,19.0,3.2,2.7,4.0,7.7,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,
5,5.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,3.8,3.6,19.0,3.2,2.7,4.0,7.7,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,
6,6.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,3.8,3.6,19.0,3.2,2.7,6.0,7.7,6.7,5.9,0.0,0.0,12.5,11.1,0.0,0.0,0.0,0.0,0.0,
7,7.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,7.7,3.6,19.0,3.2,2.7,6.0,7.7,6.7,5.9,0.0,0.0,12.5,22.2,0.0,0.0,0.0,0.0,,
8,8.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,7.7,3.6,19.0,3.2,2.7,6.0,7.7,6.7,5.9,0.0,0.0,12.5,33.3,0.0,0.0,0.0,0.0,,
9,9.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,7.7,10.5,7.7,3.6,19.0,3.2,2.7,6.0,7.7,6.7,5.9,0.0,0.0,12.5,44.4,0.0,0.0,0.0,0.0,,


### Pasar a G Sheets

In [75]:
sheet = client.open("Business_Health_Dashboard")  #open sheet

#replace sheet_name with the name that corresponds to yours, e.g, it can be sheet1
sheet = sheet.worksheet("data_cancelation_rate")

In [76]:
sheet.update([cohort_pivot.columns.values.tolist()] + cohort_pivot.values.tolist())

{'spreadsheetId': '1ZKpgLgKOSt-8JuokkfHOcTBAG72ER73-RcE-g7sttJ4',
 'updatedRange': 'data_cancelation_rate!A1:AC32',
 'updatedRows': 32,
 'updatedColumns': 29,
 'updatedCells': 928}